# 移動ライブラリ（rosnav）の使い方
rosnavという自律移動機能を使って、ロボットを指定した場所まで障害物を避けながら移動させてみましょう。

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>rosnavを使った絶対座標指定による移動をスクリプトを用いて実行する方法を学習します</p>
</div> 

## 1. 準備
自律移動のための便利な関数を登録しておきます。興味がある方は、中身をご覧ください。

In [6]:
import rospy
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import math
from tf import transformations

# ROSの初期化
rospy.init_node('send_goal')
# Actionの初期化
navclient = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
navclient.wait_for_server()

def move_to_goal(x, y, theta):
    """u指定したゴール位置・姿勢まで自律移動する
    Args:
        x : ゴールのx座標[m]
        y : ゴールのy座標[m]
        theta : ゴール時のロボットの向き[deg]
        
    Return:
        success: 成功かどうか
    """
    goal.target_pose.header.frame_id = "/map"
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    q = transformations.quaternion_from_euler(0.0, 0.0, theta / 180.0 * math.pi)

    goal.target_pose.pose.orientation.x = q[0]
    goal.target_pose.pose.orientation.y = q[1]
    goal.target_pose.pose.orientation.z = q[2]
    goal.target_pose.pose.orientation.w = q[3]

    # goal座標をロボットに送る
    navclient.send_goal(goal)
    success = navclient.wait_for_result()
    return success

In [ ]:
rvizを起動します。

In [ ]:
%%script bash --bg
rviz -d data/nav.rviz > /dev/null 2>&1

# 5. pythonスクリプトからの自律移動

## 4. 関数化
上記の処理を関数にして簡単に呼び出せるようにしましょう。

## 1. ゴール座標指定
ゴールの座標を決めましょう。rvizの右上の「Publish Point」ボタンをクリックします。

![title](./imgs/publish_point_button.png)

次に、地図上で目的地にマウスを持って行くと、rvizの左下の「Select this point」の部分に座標が表示されます。この例では、x=1.29,y=3.13,z=0.00747 が目的地の座標です。zは不要なので、x,yの値をメモしておきましょう。

![title](./imgs/point_values.png)

#### 注意: ゴールを地図から外れた位置に設定すると誤動作するので、地図内の位置にしてください。

## 2. ゴールを送る
調べた位置まで自律移動するよう、指示しましょう。

In [ ]:

success = move_to_goal(1,29, 3.13, 0)
print(success)


上の関数を使うと、以下のコマンドでx=1メートル, y=0.5メートル, theta=10度の位置姿勢に移動できます。

以下のコマンドで原点に戻ります。

In [10]:
success = move_to_goal(0, 0, 0)
print(success)

True

## 5. 応用
下記のようなことを試して見ましょう。
- ある場所についたら、次のゴールに自動的に向かうようにしてみましょう
- goalの配列を作って、数カ所を巡回させましょう
- 障害物の上にゴールを置くとどうなるでしょうか？
- 自律移動中に人を動かして、ロボットの邪魔をするとどうなるでしょうか？
- 地図の外にゴールを置くとどうなるでしょうか
